# Классификатор отзывов

*В качестве основы выбрана модель SVC с параметрами, определенными на предыдущей неделе, и обученная на корпусе отзывов на мобильные телефоны.*

## Загрузка библиотек

In [28]:
import pickle
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

## Загрузка и подготовка данных

In [29]:
def extract_info(text, label):
    try:
        if label == 1:
            text = re.findall(r'Достоинства(:.*?)Недостатки:', text)[0]
        else:
            text = re.findall(r'Недостатки(:.*?)Комментарий:', text)[0]
        return re.sub(r'\W', ' ', text)
    except:
        return text

In [4]:
train = pd.read_json('../6. Разработка сентимент-анализа под задачу заказчика/data/train.json', orient='records', lines=True).drop_duplicates()

In [30]:
train['label'] = train['rating'].apply(lambda x: int(x == 5))
train['length'] = train['text'].apply(lambda x: len(x))
train['text_raw'] = train['text']
train['text'] = train.apply(lambda x: extract_info(x['text'], x['label']), axis=1)

In [10]:
train = shuffle(pd.concat([train[train['label'] == 0], 
                           train[train['label'] == 1].sort_values('length', ascending=False).head(1680)]), 
                           random_state=777)

## Вспомогательные функции

In [11]:
def make_pipeline(vectorizer, transformer, classifier):
    return Pipeline([
            ('vectorizer', vectorizer),
            ('transformer', transformer),
            ('classifier', classifier)
        ])

In [14]:
def make_classifier(text, label):
    svc = make_pipeline(CountVectorizer(min_df=10, ngram_range=(1, 2), max_df=0.85, stop_words=None),
                        TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=True, use_idf=True),
                        LinearSVC(max_iter=200, loss='hinge', C=0.9, tol=0.0001, random_state=777))
    svc.fit(text, label)
    return svc

## Тренировка классификатора

In [33]:
texts = train['text'].values
labels = train['label'].values

In [34]:
classifier = make_classifier(texts, labels)
classifier

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.85, max_features=None, min_df=10,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
     ...nge', max_iter=200, multi_class='ovr',
     penalty='l2', random_state=777, tol=0.0001, verbose=0))])

In [35]:
accuracy_score(labels, classifier.predict(texts))

0.9779761904761904

In [36]:
with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)

## Тестирование классификатора

In [37]:
classifier = joblib.load('sentiment_classifier.pkl')

In [38]:
classifier

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.85, max_features=None, min_df=10,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
     ...nge', max_iter=200, multi_class='ovr',
     penalty='l2', random_state=777, tol=0.0001, verbose=0))])

In [41]:
print(f"Отзыв:\n{train.iloc[100, 4]}")
print(f"Истинная тональность: {labels[100]}")
print(f"Предсказанная тональность: {classifier.predict([train.iloc[100, 4]])[0]}")

Отзыв:
Достоинства: - красивый - большой яркий дисплей - мне зарядки хватает на 2 суток Недостатки: - для женской руки не удобны наличия сенсорных кнопок нажимаешь их ладонью держа одной рукой например в метро в этом плане удобнее телефоны  с одной только центральной кнопкой не сенсорной Комментарий: 6 лет ходила с айфонами 4-5эс-се и почувствовала, что начинает портиться зрение от небольшого дисплея но покупать айфон мимнимум за 50 не вижу смысла. Выбрала данную модель по новизне размеру экрана и бренду. Классный экран глаза отдыхают практически, нет необходимости везде ходить с зарядкой как с айфоном, но андроид....вообще это еще один плюс что теперь мое зрение меньше страдает потому что я стала  меньше пользоваться телефоном потому что чтобы привыкнуть к новому интерфейсу сразу не могу, но верю в себя)))
Истинная тональность: 1
Предсказанная тональность: 1


In [42]:
print(f"Отзыв:\n{train.iloc[50, 4]}")
print(f"Истинная тональность: {labels[50]}")
print(f"Предсказанная тональность: {classifier.predict([train.iloc[50, 4]])[0]}")

Отзыв:
Достоинства: 1. Красивый 2. Тач  отпечатков пальцев работает идеально 3. В принципе  мощный на проги и игры Недостатки: 1.ЭКРАН!!!!Безумно слабый. Стоит уронить-прощайтесь.(ремонт в официальном салоне стоит 9,5 тысяч так-то) 2.Скользкий без чехла 3. Звук не особо 4. Мега камера? Вы серьёзно? Снимает не лучше 8 мег., хуже айфра и сяоми. Комментарий: не стоит 20+к.
Истинная тональность: 0
Предсказанная тональность: 0


In [44]:
classifier.predict(["Достаточно неплохой агрегат"])[0]

1